## NOTEBOOK: 04_gold_aggregations
#
# GOAL:
  Turn clean "Silver" data into "Gold" tables that are ready for dashboards and analytics.
#
### INPUT:
- silver_prices  (one row per asset per pull)

# ##OUTPUT:
  - gold_hourly_prices  (hourly average per asset)
  - gold_hourly_movers  (% change hour-over-hour per asset)

### WHY GOLD?
-  Gold is optimized for fast queries in dashboards.
-   Instead of calculating aggregates every time, we precompute
-   them once and store results in Delta tables. 

In [0]:
# Set catalog and schemas
spark.sql("USE CATALOG databricks_cata")
spark.sql("USE SCHEMA price_movers")

spark.sql("SELECT current_catalog(), current_schema()").show(truncate=False)

In [0]:
# Read Silver and do sanity checks

from pyspark.sql import functions as F

silver = spark.table("silver_prices")

silver.select("asset_id", "event_ts", "event_ts_sa", "price").orderBy(F.col("event_ts").desc()).show(5, truncate=False)


### Create hourly price table


In [0]:
# Hourly bucket based on event_ts (source time)
# Create an hourly time bucket using event_ts.
# This groups data into hourly windows like:
# 2026-01-17 15:00:00
# 2026-01-17 16:00:00

#For eacg (asset_id, hour) we compute:
# avg_price
# min_price
# max_price
# samples

# This table makes dashboars fast

hourly = (
    silver
    .withColumn("hour_ts_sa", F.date_trunc("hour", F.col("event_ts_sa")))
    .groupBy("asset_id","vs_currency", "hour_ts_sa")
    .agg(
        F.avg("price").alias("avg_price"),
        F.min("price").alias("min_price"),
        F.max("price").alias("max_price"),
        F.count("*").alias("samples")
    )
)

hourly.orderBy(F.col("hour_ts_sa").desc(), F.col("asset_id")).show(20, truncate=False)

### Write gold_hourly_prices (Delta Table)

In [0]:
hourly.write.format("delta").option("mergeSchema", "true").mode("overwrite").saveAsTable("gold_hourly_prices")

print("Wrote Delta table: gold_hourly_prices")

### Compte hour-over-hour mover (percentage changes)

For each asset, compare the current hour avg_price to the previous hour avg_price.

pct_change = ((current - previous) / previous) * 100

This will show most moving -> least moving on the dashboard


In [0]:
from pyspark.sql.window import Window

gold_hourly = spark.table("gold_hourly_prices")

# Window lets us look at the previous row in time for each asset
w = Window.partitionBy("asset_id","vs_currency").orderBy("hour_ts_sa")

movers = (
    gold_hourly
    .withColumn("prev_avg_price", F.lag("avg_price").over(w))
    .withColumn(
        "pct_change",
        F.when(F.col("prev_avg_price").isNull(), None)
        .otherwise((F.col("avg_price") - F.col("prev_avg_price")) / F.col("prev_avg_price") * 100)
    )

    # We only keep rowa where we have a previous hour to compare to
    .where(F.col("prev_avg_price").isNotNull())
)
# Preview sorted by latest hour and biggest moves
movers.orderBy(F.col("hour_ts_sa").desc(), F.col("pct_change").desc()).show(10, truncate = False)

### Write gold_hourly_movers (Delta Table)

- Save movers as a Gold table for the dashboard
- This is what the Top Movers visual will query

In [0]:
movers.write.format("delta").option("mergeSchema", "true").mode("overwrite").saveAsTable("gold_hourly_movers")
print("Wrote Delta Table: gold_hourly_movers")

### Validate Gold tables

In [0]:
spark.sql("SELECT COUNT(*) AS hourly_rows FROM gold_hourly_prices").show()
spark.sql("SELECT COUNT(*) AS movers_rows FROM gold_hourly_movers").show()

# Show latest movers

spark.sql("""
SELECT asset_id, vs_currency, hour_ts_sa, avg_price, prev_avg_price, pct_change
FROM gold_hourly_movers
ORDER BY hour_ts_sa DESC, pct_change DESC
LIMIT 20
""").show(truncate=False)